# Clean Data for Analysis

In [1]:
import pandas as pd

pd.set_option('display.max_columns', 150)
pd.options.display.float_format = '{:,.3f}'.format

In [2]:
# From process_data.ipynb, we have the following:
# geo_parcels: parcel boundaries and cities
# digest_full: all digest data where LUC == 101
# sales_full: all sales data LUC == 101

DATA_PATH = 'output/'
geo_parcels = pd.read_csv(DATA_PATH + 'parcels_geo.csv')
digest_full = pd.read_parquet(DATA_PATH + 'digest_full.parquet')
sales_full = pd.read_parquet(DATA_PATH + 'sales_full.parquet')

In [3]:
# Adjust diff for inflation, using CPI-U from the
# BLS: https://data.bls.gov/timeseries/CUUR0000SA0
# Use December CPI for each year as demoninator, and
# 2022 December CPI as numerator
december_2022_cpi = 296.797
december_cpi_year = {
    2010: 219.179,
    2011: 225.672,
    2012: 229.601,
    2013: 233.049,
    2014: 234.812,
    2015: 236.525,
    2016: 241.432,
    2017: 246.524,
    2018: 251.233,
    2019: 256.974,
    2020: 260.474,
    2021: 278.802,
    2022: 296.797
}
inflation_scalar = {x: december_2022_cpi / december_cpi_year[x] for x in december_cpi_year}
print(f"Inflation scalars: {inflation_scalar}")

Inflation scalars: {2010: 1.3541306420779364, 2011: 1.3151698039632742, 2012: 1.2926642305564873, 2013: 1.2735390411458536, 2014: 1.2639771391581351, 2015: 1.25482295740408, 2016: 1.2293192286026708, 2017: 1.2039274066622319, 2018: 1.1813615249589027, 2019: 1.1549689851891634, 2020: 1.1394496187719314, 2021: 1.0645440133141082, 2022: 1.0}


## Parcel Cleaning
- Drop duplicate TAXYR, PARID keys (parcels that contain multiple structures, these are essentially ADUs since we are only looking at single-family); retain only the row with the largest living area square footage (main structure).

Only keep one record per parcel per year (building with greatest SF)

In [4]:
init_len = len(digest_full)
digest_full = digest_full.sort_values(by="sqft_living").drop_duplicates(subset=["TAXYR", "PARID"], keep="first")

print(f"Number of dropped duplicates: {init_len - len(digest_full)}")
print(f"Number of total parcels (all years): {len(digest_full)}")

Number of dropped duplicates: 26676
Number of total parcels (all years): 2739296


Drop rows without owner address

In [5]:
# Drop rows without an owner address
init_len = len(digest_full)
digest_full = digest_full[digest_full["Owner Adrstr"] != ""]
print(f"Number of empty addresses dropped: {init_len - len(digest_full)}")

Number of empty addresses dropped: 0


In [6]:
count_of_records_parcel = pd.DataFrame(digest_full.groupby(by="PARID")["PARID"].count()).rename(
    columns={"PARID": "count_records"}
)
print(f"Total number of unique parcels: {len(count_of_records_parcel)}")

count_missing_a_record = (count_of_records_parcel["count_records"] < 13).sum()
print(f"Nun parcels without a record for every year in period: {count_missing_a_record}")
print("")

tot_parcels_begin = digest_full[digest_full["TAXYR"] == 2010].merge(
    count_of_records_parcel,
    on="PARID",
    how="inner"
)

begin_and_missing = (tot_parcels_begin["count_records"] < 13).sum()
tot_count_missing = tot_parcels_begin["PARID"].count() - begin_and_missing
print(f"Total number of parcels around at start of study period: {len(tot_parcels_begin)}")
print(f"Num of parcels around at start that don't have an entry for every year: {begin_and_missing}")

Total number of unique parcels: 226759
Nun parcels without a record for every year in period: 39817

Total number of parcels around at start of study period: 205575
Num of parcels around at start that don't have an entry for every year: 18633


### Adjust Aprtot for inflation

In [7]:
digest_full["Aprtot_adj"] = digest_full["Aprtot"] * digest_full["TAXYR"].map(inflation_scalar)
digest_full[["TAXYR", "Aprtot", "Aprtot_adj"]].sample(3)

,TAXYR,Aprtot,Aprtot_adj
1724835,2016,"621,300.000","763,776.037"
1664332,2015,"363,400.000","456,002.663"
159049,2010,"85,000.000","115,101.105"


## Sales Cleaning
- Drop non-arms length transcations (Saleval codes = D, T, G, 4, 4A, 4B, 4C, 4D, 4E, 6F), price under 1K
- Only keep one record when a parcel has been sold twice in one day, with preference for keeping record with Saleval == M. If a parcel had multiple sales in one day, we assume only one of these may have been a market sale and the rest were transfers (manual inspection appears to reveal this).
- Replace sales price for multi-parcel sales with sales price / number of parcels in that transcation. Sales price is total transcation value, which includes many parcels for a multi-parcel sale. 

Saleval code distribution before cleaning

In [8]:
sales_full["Saleval"].value_counts().head(10)

Saleval
0     117330
T      36730
G      23159
5      17287
M      13777
9      13131
3      10205
RE      8124
4       7015
4E      6112
Name: count, dtype: Int64

Drop non-arms length transactions
- Saleval codes: D, T, G, 4, 4B, 4C, 6B, 6F
- Any remaining sales where sales price is under 1K

In [9]:
drop_saleval = ["D", "T", "G", "4", "4A", "4B", "4C", "4D", "4E", "6F"]
init_len = len(sales_full)
sales_full["Saleval"] = sales_full["Saleval"].str.strip()
sales_full = sales_full[~sales_full["Saleval"].isin(drop_saleval)]
sales_full = sales_full[sales_full["SALES PRICE"] > 1000]
print(f"Number of dropped records: {init_len - len(sales_full)}")
print(f"Number of total sales records: {len(sales_full)}")

Number of dropped records: 85636
Number of total sales records: 189717


Examine rows where PARID and Saledt are identical (e.g. potentially duplicate records)

In [10]:
sales_full[sales_full["PARID"] == "06 031200020099"]

,TAXYR,PARID,Luc,Saledt,SALES PRICE,FAIR MARKET VALUE,DEED TYPE,Costval,Saleval,GRANTOR,GRANTEE
26787,2012,06 031200020099,101,03-MAY-2011,"192,400.000","85,700.000",DP,85700,5,MATASSINO JOSEPH A JR,PNC BANK N A
26788,2012,06 031200020099,101,03-MAY-2011,"192,400.000","85,700.000",SW,85700,2,PNC BANK N A,FEDERAL HOME LOAN MORTGAGE CORP


Only keep one record where PARID and Saledt. Priority to keep record where Saleval == M.

If a sale happened for the same parcel twice in one day, the assumption that the second sale is non-arms length (e.g. a transfer to some other entity)

In [11]:
init_len = len(sales_full)

# Sort values to keep records where Saleval is M first (in the case of duplicates being dropped)
sales_full["M_flag"] = sales_full["Saleval"].apply(lambda x: True if x == "M" else False)
sales_full = sales_full.sort_values(by=["M_flag"], ascending=False)
sales_full = sales_full.drop_duplicates(subset=["PARID", "Saledt"], keep="first")

print("Count dropped where sale was identical except for Saleval: ", init_len - len(sales_full))

Count dropped where sale was identical except for Saleval:  7679


For multi-parcel sales, the sales price needs to be replaced with sales price divided by number of parcels in the transcation. We consider a record as part of the same multi-parcel sale when Saleval flag == M, Saledt, GRANTEE, and Sales Price are the same.

Drop any records where Saleval == M but there is only one record in that transcation. This indicates it might've been a multi-parcel sale with parcels located outside of Fulton. We do not have complete records for these, and they are an negligible total.

In [12]:
sales_full['sale_size'] = sales_full.groupby(by=["Saledt", "SALES PRICE", "GRANTEE"])["PARID"].transform('count')
sales_full["sales_price"] = sales_full["SALES PRICE"] / sales_full['sale_size']

init_len = len(sales_full)
sales_full = sales_full[~((sales_full["Saleval"] == "M") & (sales_full["sale_size"] == 1))]
print(f"Number of invalid multi-parcel sales dropped: {init_len - len(sales_full)}")

Number of invalid multi-parcel sales dropped: 1346


Verify multi-parcel sale calculation

In [13]:
sales_full[sales_full["Saleval"] != "M"].sample(2)

,TAXYR,PARID,Luc,Saledt,SALES PRICE,FAIR MARKET VALUE,DEED TYPE,Costval,Saleval,GRANTOR,GRANTEE,M_flag,sale_size,sales_price
87024,2014,22 511010401008,101,09-AUG-2013,"200,000.000","183,100.000",WD,"183,100",0,MILLER MARY,COLE SHARON C,False,1,"200,000.000"
126205,2017,22 473106220158,101,04-JAN-2016,"400,000.000","222,400.000",LW,"256,100",3,DASHER ROBIN L,SPOLZINO RICHARD DAVID &,False,1,"400,000.000"


In [14]:
sales_full[sales_full["Saleval"] == "M"].sample(2)

,TAXYR,PARID,Luc,Saledt,SALES PRICE,FAIR MARKET VALUE,DEED TYPE,Costval,Saleval,GRANTOR,GRANTEE,M_flag,sale_size,sales_price
144775,2018,14 019400070695,101,06-APR-2017,"132,000.000","257,500.000",LW,"257,500",M,FAIRVIEW VENTURES LLC,D R HORTON CROWN LLC,True,3,"44,000.000"
253976,2022,13 016100050391,101,23-JUN-2021,"3,036,993.000","166,800.000",LW,"195,400",M,ATL 2 SF LLC,ATL 2 SF LLC,True,11,"276,090.273"


In [15]:
sales_full[
    (sales_full["Saledt"] == "10-MAY-2017")
    & (sales_full["SALES PRICE"] == 265500)
]

,TAXYR,PARID,Luc,Saledt,SALES PRICE,FAIR MARKET VALUE,DEED TYPE,Costval,Saleval,GRANTOR,GRANTEE,M_flag,sale_size,sales_price
137527,2018,14 003700010056,101,10-MAY-2017,"265,500.000","33,000.000",WD,"33,000",M,GCM SOUTHEASTERN LLC,BUDGET HOME RENTALS LLC,True,5,"53,100.000"
138501,2018,14 006000030336,101,10-MAY-2017,"265,500.000","43,900.000",WD,"43,900",M,GCM SOUTHEASTERN LLC,BUDGET HOME RENTALS LLC,True,5,"53,100.000"
137567,2018,14 003800030772,101,10-MAY-2017,"265,500.000","43,700.000",WD,"43,700",M,GCM SOUTHEASTERN LLC,BUDGET HOME RENTALS LLC,True,5,"53,100.000"
137576,2018,14 003800040292,101,10-MAY-2017,"265,500.000","46,000.000",WD,"46,000",M,GCM SOUTHEASTERN LLC,BUDGET HOME RENTALS LLC,True,5,"53,100.000"
137586,2018,14 003800050192,101,10-MAY-2017,"265,500.000","34,800.000",WD,"34,800",M,GCM SOUTHEASTERN LLC,BUDGET HOME RENTALS LLC,True,5,"53,100.000"


Saleval codes after cleaning

In [16]:
print(f"Number of total sales records: {len(sales_full)}")
sales_full.groupby("Saleval")["Saleval"].count().sort_values(ascending=False).head(10)

Number of total sales records: 180692


Saleval
0     116433
9      12515
5      11706
3       9586
RE      8034
M       6491
2       4642
W       3515
6       1165
5A      1134
Name: Saleval, dtype: int64

### Adjust sales price, fmv for inflation

In [17]:
sales_full["price_adj"] = sales_full["sales_price"] * (sales_full["TAXYR"] - 1).map(inflation_scalar)
sales_full["fmv_adj"] = sales_full["FAIR MARKET VALUE"] * (sales_full["TAXYR"] - 1).map(inflation_scalar)
sales_full[["TAXYR", "sales_price", "price_adj", "FAIR MARKET VALUE", "fmv_adj"]].sample(3)

,TAXYR,sales_price,price_adj,FAIR MARKET VALUE,fmv_adj
131101,2018,"297,000.000","357,566.440","265,600.000","319,763.119"
114186,2017,"72,460.000","89,076.471","181,200.000","222,752.644"
145778,2018,"96,439.000","116,105.555","73,800.000","88,849.843"


## Save

In [18]:
OUTPUT_PATH = 'output/'

digest_full.to_parquet(OUTPUT_PATH + 'digest_full_clean.parquet')
sales_full.to_parquet(OUTPUT_PATH + 'sales_full_clean.parquet')